# SASによるエージェント
[langchainのconversationalエージェント](https://python.langchain.com/docs/modules/agents/agent_types/chat_conversation_agent)の構成を参考にしたエージェントの実行例です。  
エージェントは別にsasファイルとして保存しているものを読み込みます。

会話の1往復の間に必要に応じてツールを一度使い、その情報を使って回答することができます。  
簡単な例でのテストなので、入出力の内容に応じて例外処理やエスケープ処理など適宜追加してください。

## エージェントの構成

- llm chain  
  各情報からプロンプトを生成し、LLMを呼び出す
- perser  
  出力情報を処理する  
- memory  
  過去の会話の履歴を保存、出力する
- executor  
  ツールを実行し、結果を途中経過として保存する

## モデル・エージェントのパラメータ
apiキーやモデルについては公式ページを参照してください。

実行時のgpt-3.5-turboのバージョンは`0613`です。再現したい場合など必要に応じて指定を追加してください。

その後のパラメータはlangchainにあわせて設定しています。  
langchianのデフォルトのテンプレートをマクロ変数等に修正したものを付属しています。

In [ ]:
/* API Key  */
%let api_key = OpenAI API key ;
 
%let llm = "model":"gpt-3.5-turbo", "temperature":0 ;

/* labels of role */
%let ai_prefix = AI ;
%let human_prefix = Human ;

/* filenames to be referenced */
%let prefix = prefix ;
%let suffix = suffix ;
%let tools = tools ;
%let format_instraction = fmt_inst ;
%let chat_history = chat_h ;
%let agent_scratchpad = agent_sp ;

/* template files */
filename prefix temp ;
filename suffix temp ;
filename fmt_inst temp ;

以下は[エージェントのデフォルトのプロンプト](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/agents/conversational/prompt.py)の内容をマクロ変数やファイル読み込み等にしてファイルに出力しています。  
マクロ変数を展開させたくない場合はデータステップにより作成しています。

In [ ]:
proc stream outfile= prefix prescol ;
begin &streamdelim.;
Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------
Assistant has access to the following tools:
;;;;
run ;

data _null_ ;
 file fmt_inst ;
 infile cards4 ;
 input ;
  len = lengthn(_infile_) ;
  put _infile_ $varying. len ;
cards4 ;
To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [&tool_names.]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:
```
Thought: Do I need to use a tool? No
{ai_prefix}: [your response here]
```
;;;;
run ;
 
data _null_ ;
  file suffix ;
  infile cards4 ;
  input ;
  len = lengthn(_infile_) ;
  put _infile_ $varying. len ;
cards4 ;
Begin!

Previous conversation history:
&streamdelim. readfile &chat_history.;
New input: &input.
&streamdelim. readfile &agent_scratchpad.;
;;;;
run ;

## ツールの設定

マクロ変数にツール名の一覧と、各ツールの情報を設定します。  
以下ではサンプルに現在の日時を取得するツール(SASマクロ)を設定しています。

マクロではパラメータが無い場合でもNoneと返されるためダミーのパラメータを設定し、最終的に`observation`というマクロ変数に値を格納するようにします。

In [ ]:
%let tool_names = get_datetime ;
filename tools temp ;

data _null_ ;
  file tools ;
  infile cards4 ;
  input ;
  put _infile_  ;
cards4 ;
{
  "name": "get_datetime",
  "description": "Get the current datetime in iso8601 format",
  "parameters": None
}
;;;;
run ;

/* sample tool */
%macro get_datetime(dummy_param) ;
  data _null_ ;
    datetime = put(datetime(), e8601dt.) ;
    call symputx("observation", datetime) ;
  run ;
%mend ;

## チャット履歴

履歴は扱いやすいようデータセットに保存するようにしています。  
変数の順に会話履歴に追加されます。マクロ変数のai_prefix、human_prefixとあわせた名前にしておいてください。  
必要に応じて、先に過去の履歴を設定することもできます。   

In [ ]:
/* 
履歴ありの場合
data work.chat_history ;
  infile cards dsd missover ;
  length HUMAN AI $1000 ;
  input HUMAN AI ;
cards ;  
hi,Hello! How can I assist you today?
;
run ;
*/
  
data work.chat_history ;
  length HUMAN AI $1000 ;
  call missing(of _all_) ;
  stop ;
run ;

## サンプル実行

インプットは比較的短いものを想定し、マクロ変数としています。  
ツールを使わないケースと使うケースを実行し、履歴を出力します。  
上記のデフォルトのプロンプトでは出力が形式が想定した形式にならないことがあることから、実行時には一部修正したプロンプトを使用しています。  
https://github.com/langchain-ai/langchain/issues/1358  
修正したプロンプトについてはひとまず非公開としています。

In [6]:

%include "agent.sas" ;

%run_agent(input =hi., dsmemory = work.chat_history) ;
%run_agent(input =What is the date and time now?, dsmemory = work.chat_history) ;

proc print data = work.chat_history ;
run ;

OBS,HUMAN,AI
1,hi.,Hello! How can I assist you today?
2,What is the date and time now?,The current date and time is 2023-08-29 18:12:48. How can I assist you further?


## デバッグ

必要に応じて、filenameステートメントで参照されている以下のファイルの内容を確認してください。

- pprint：読みやすい状態の生成されたプロンプト
- payload：送信するjson
- content：LLMからの回答